<a href="https://colab.research.google.com/github/o-semenkova/ml_home_tasks/blob/main/%D0%9C%D1%83%D0%BB%D1%8C%D1%82%D0%B8%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D1%82%D0%B0_%D0%BC%D1%83%D0%BB%D1%8C%D1%82%D0%B8%D0%B7%D0%BD%D0%B0%D1%87%D0%BD%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Для мультикласової класифікації будемо використовувати набір даних про [ризик ожиріння](https://www.kaggle.com/datasets/ikjotsingh221/obesity-risk-prediction-cleaned/data).

Цей датасет взятий з бібліотеки UCI. Він був очищений за допомогою таких технік: нормалізація за допомогою z-score, one-hot кодування, видалення викидів, масштабування min-max і відбір ознак.

Стовпчик цільових значень 'NObeyesdad' містить наступне кодування рівнів ожиріння:

- Insufficient_Weight: 0
- Normal_Weight: 1
- Overweight_Level_I: 2
- Overweight_Level_II: 3
- Obesity_Type_I: 4
- Obesity_Type_II: 5
- Obesity_Type_III: 6

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

In [ ]:
# Завантажимо дані
df = pd.read_csv('estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.csv')

In [ ]:
df.dtypes

Height                            float64
Weight                            float64
family_history_with_overweight      int64
SCC                                 int64
MTRANS_Walking                      int64
FAVC_z                            float64
FCVC_minmax                       float64
NCP_z                             float64
CAEC_minmax                       float64
CH2O_minmax                       float64
FAF_minmax                        float64
TUE_z                             float64
CALC_z                            float64
Age_bin_minmax                    float64
NObeyesdad                          int64
dtype: object

In [ ]:
target_col_name = 'NObeyesdad'
X = df.drop(columns=target_col_name)
y = df[target_col_name]

# Розділимо дані на тренувальні та тестові набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# Логістична регресія зі стратегією one-vs-rest (OvR)
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train, y_train)
ovr_predictions = ovr_model.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу
print(classification_report(y_test, ovr_predictions))

              precision    recall  f1-score   support

           0       0.83      0.93      0.87        82
           1       0.61      0.42      0.50        85
           2       0.61      0.54      0.57        87
           3       0.42      0.45      0.43        83
           4       0.64      0.58      0.61       103
           5       0.78      0.82      0.80        89
           6       0.79      0.99      0.88        97

    accuracy                           0.68       626
   macro avg       0.67      0.68      0.67       626
weighted avg       0.67      0.68      0.67       626



In [ ]:
ovr_model.classes_

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
ovr_model.predict_proba(X_test)[:10].round(2)

array([[0.  , 0.  , 0.02, 0.02, 0.12, 0.08, 0.75],
       [0.41, 0.27, 0.25, 0.04, 0.03, 0.  , 0.  ],
       [0.  , 0.01, 0.02, 0.02, 0.07, 0.16, 0.72],
       [0.  , 0.03, 0.12, 0.14, 0.64, 0.04, 0.02],
       [0.  , 0.2 , 0.3 , 0.2 , 0.27, 0.02, 0.  ],
       [0.  , 0.05, 0.09, 0.15, 0.55, 0.16, 0.  ],
       [0.  , 0.1 , 0.17, 0.46, 0.23, 0.04, 0.  ],
       [0.  , 0.01, 0.05, 0.07, 0.19, 0.65, 0.03],
       [0.35, 0.35, 0.05, 0.13, 0.12, 0.  , 0.  ],
       [0.  , 0.06, 0.23, 0.14, 0.31, 0.26, 0.  ]])

In [ ]:
# Логістична регресія зі стратегією one-vs-one (OvO)
ovo_model = OneVsOneClassifier(log_reg)
ovo_model.fit(X_train, y_train)
ovo_predictions = ovo_model.predict(X_test)

# Обчислимо метрики precision та recall для кожного класу
print(classification_report(y_test, ovo_predictions))


              precision    recall  f1-score   support

           0       0.86      0.94      0.90        82
           1       0.82      0.64      0.72        85
           2       0.74      0.80      0.77        87
           3       0.67      0.64      0.65        83
           4       0.81      0.80      0.80       103
           5       0.92      0.89      0.90        89
           6       0.88      0.99      0.93        97

    accuracy                           0.82       626
   macro avg       0.81      0.81      0.81       626
weighted avg       0.82      0.82      0.81       626



Бачимо, що стратегія OVO спрацювала тут краще!

Як ми б могли порахувати усереднені метрики окремо

In [ ]:
# Усереднені метрики macro та micro для OvR
ovr_macro_precision = precision_score(y_test, ovr_predictions, average='macro')
ovr_micro_precision = precision_score(y_test, ovr_predictions, average='micro')
ovr_macro_recall = recall_score(y_test, ovr_predictions, average='macro')
ovr_micro_recall = recall_score(y_test, ovr_predictions, average='micro')

# Усереднені метрики macro та micro для OvO
ovo_macro_precision = precision_score(y_test, ovo_predictions, average='macro')
ovo_micro_precision = precision_score(y_test, ovo_predictions, average='micro')
ovo_macro_recall = recall_score(y_test, ovo_predictions, average='macro')
ovo_micro_recall = recall_score(y_test, ovo_predictions, average='micro')

# Створимо датафрейм для відображення результатів
results = pd.DataFrame({
    'Metric': ['Macro Precision', 'Micro Precision', 'Macro Recall', 'Micro Recall'],
    'OvR': [ovr_macro_precision, ovr_micro_precision, ovr_macro_recall, ovr_micro_recall],
    'OvO': [ovo_macro_precision, ovo_micro_precision, ovo_macro_recall, ovo_micro_recall]
})
print(results)

            Metric       OvR       OvO
0  Macro Precision  0.666983  0.813241
1  Micro Precision  0.678914  0.816294
2     Macro Recall  0.675544  0.812988
3     Micro Recall  0.678914  0.816294


# Мультилейбл класифікація

OneVsRestClassifier може бути використаний також для задач мультилейбл класифікації.

In [ ]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_multilabel_classification

# Створення мультикласового датасету
X, y = make_multilabel_classification(n_samples=100, n_features=20, n_classes=3, n_labels=2, random_state=42)

# OneVsRestClassifier
ovr_clf = OneVsRestClassifier(LogisticRegression())
ovr_clf.fit(X, y)
ovr_multilable_predictions = ovr_clf.predict(X)
ovr_multilable_proba_predictions = ovr_clf.predict_proba(X)

Тут може бути таке, що ми передбачаємо 0 для всіх класів. Тому рекомендую все ж самостійно налаштувати трешхолд в задачі мультилейбл класифікації.

In [ ]:
ovr_multilable_predictions[:4]

array([[0, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 1, 0]])

In [ ]:
ovr_multilable_proba_predictions.round(2)[:5]

array([[0.04, 1.  , 0.66],
       [0.12, 0.03, 0.38],
       [0.5 , 0.85, 0.8 ],
       [0.03, 1.  , 0.09],
       [0.02, 1.  , 0.03]])

In [ ]:
print(classification_report(y, ovr_multilable_predictions))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89        45
           1       0.93      0.96      0.94        70
           2       0.95      0.95      0.95        59

   micro avg       0.93      0.94      0.93       174
   macro avg       0.92      0.93      0.93       174
weighted avg       0.93      0.94      0.93       174
 samples avg       0.84      0.82      0.82       174



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y

array([[0, 1, 0],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 0],
       [0, 1, 0],
       [1, 1, 0],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 1, 1],
       [0, 1, 1],
       [1, 1, 1],
       [1, 0, 1],
       [0, 1, 1],
       [1, 1, 0],
       [0, 1, 1],
       [1, 0, 1],
       [0, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 1],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [0, 1, 1],
       [1, 1, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 0, 1],
       [1, 0, 1],
       [1, 1, 1],
       [0, 1, 1],
       [0, 0, 0],
       [1, 1, 1],
       [1, 0, 1],
       [0, 0, 0],
       [0, 1, 1],
       [1, 1, 1],
       [0, 1, 1],
       [0, 0, 0],
       [1, 1, 0],
       [0, 1, 1],
       [0, 1, 0],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [0,